In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [47]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

Label encoding per il "genere", che puo essere solo Male e Female per cui 0 e 1

In [48]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

La colonna delle nazioni viene invece trasformata in una combinazione di 0 e 1

In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Lo scaling è assolutamente necessario in un network neurale. Tutti i parametri vengono scalati sempre.

In [51]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

add() aggiunte un layer. Se ne possono usare quanti layer si vogliono.

Dense rappresenta un fully connected layer

units indica quanti neuroni sono presenti nel layer. Non c'è una regola ma la scelta proviene da test e da esperienza.

In [52]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # hidden layer 1
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # hidden layer 2

Visto che l'output che dobbiamo ottenere è un booleano, l'ultimo layer avrà solo un neurone.

Come funzione di attivazione viene usato un sigmoide in modo da avere sia una classificazione (0 o 1) sia una probabilità che il dato appartenga effettivamente all'insieme 0 o 1.

Nel caso di una classificazione non binaria, viene utilizzata l'activation "softmax"

In [53]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # output layer

Un ottimo metodo di ottimizzazione è lo Stochastic Gradient Descent, che corrisponde all'ottimizzatore "adam".

Nel caso in cui si debba ricavare un risultato binario/booleano, la funzione di loss deve sempre essere "binary_crossentropy". Per una classificazione non binaria si utilizza "categorical_crossentropy".

metrics è una lista di metriche da computare per valutare la performance del modello, ma non vengono usate per l'effettivo training (solo la loss function viene usata)

In [54]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

batch_size indica ogni quanto computare il loss confrontando i valori in output dal network con i valori reali (y_train).

Confrontare ad ogni entry sarebbe troppo lento, quindi il confronto del loss e ricalcolo dei pesi viene fatto ogni 32 entry.

In [55]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

In [63]:
x_pred = np.array([[600, "France", "Male", 40, 3, 60000, 2, 1, 1, 50000]])
x_pred[:, 2] = le.transform(x_pred[:, 2])
x_pred = np.array(ct.transform(x_pred))
x_pred = sc.transform(x_pred)
y_pred = ann.predict(x_pred)
print(y_pred*100, "%")
print(y_pred > 0.5)

[[4.005411]] %
[[False]]


In [64]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), axis=1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [65]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1510   85]
 [ 194  211]]


0.8605